# Use Vgg16 model with Keras

[State Farm Distracted Driver Detection | Kaggle](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data)

In [ ]:
%pwd

In [2]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

In [3]:
DATA_HOME_DIR = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/"

In [ ]:
%cd $DATA_HOME_DIR

In [ ]:
%cd train

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
# create c0 .. c9 in train/valid directory
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [ ]:
from shutil import copyfile

In [ ]:
# copy c?/*.jpg to sample/train/c?/*.jpg
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%cd ../../..

## train data with pre-trained model

In [9]:
import numpy as np
import pandas as pd
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

In [4]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [5]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model


def finetune(model, num_classes):
    # remove last layer
    model.pop()
    # set all layers untrainable.
    for layer in model.layers: layer.trainable=False
    # add new layer
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def get_batches(path, dirname, gen=image.ImageDataGenerator(), shuffle=True,
                batch_size=64, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

get pre-trained weight from fast.ai server http://www.platform.ai/models/

```wget http://www.platform.ai/models/vgg16.h5```

You can also download [here](https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view)

In [14]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.N)

In [13]:
model = VGG_16()

# load pre-trained weights!!!
model.load_weights('vgg16.h5')

# remove last layer and add new layer
# ftmodel = finetune(model,10)

model.pop()
model.add(Dense(10, activation='softmax'))
layers = model.layers
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
for layer in layers[12:]: layer.trainable=True
K.set_value(opt.lr, 0.001)


model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 3, 224, 224)   0           lambda_input_5[0][0]             
____________________________________________________________________________________________________
zeropadding2d_53 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_5[0][0]                   
____________________________________________________________________________________________________
convolution2d_53 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_53[0][0]           
____________________________________________________________________________________________________
zeropadding2d_54 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_53[0][0]           
___________________________________________________________________________________________

In [17]:
batch_size=64
# path = DATA_HOME_DIR
path = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/sample/"

In [18]:
batches = get_batches(path,'train', batch_size=batch_size)
val_batches = get_batches(path,'valid', batch_size=batch_size)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [ ]:
# train finetuned model(only last layer)
no_of_epochs=1

for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    ftmodel.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
    latest_weights_filename = 'ft%d.h5' % epoch
    ftmodel.save_weights(latest_weights_filename)

In [6]:
ftmodel.load_weights("ft0.h5")

In [19]:
fit_model(model, batches, val_batches, 4)

Epoch 1/4
1500/1500 [==============================] - 166s - loss: 14.0808 - acc: 0.0993 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/4
1500/1500 [==============================] - 166s - loss: 14.4740 - acc: 0.1020 - val_loss: 14.5869 - val_acc: 0.0950
Epoch 3/4
1500/1500 [==============================] - 165s - loss: 14.4740 - acc: 0.1020 - val_loss: 14.4902 - val_acc: 0.1010
Epoch 4/4
1500/1500 [==============================] - 166s - loss: 14.4740 - acc: 0.1020 - val_loss: 14.5708 - val_acc: 0.0960


In [20]:
model.save_weights("ftvgg_train12.h5")

## test data

In [22]:
test_batches = get_batches(DATA_HOME_DIR, 'test', batch_size=2*batch_size, class_mode=None)
preds = model.predict_generator(test_batches, test_batches.nb_sample)

Found 79726 images belonging to 1 classes.


In [ ]:
#df = pd.read_csv(DATA_HOME_DIR+"submission2.csv")
#classes = sorted(batches.class_indices, key=batches.class_indices.get)
#preds = df[classes].values

In [23]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)
preds = do_clip(preds,0.93)

In [24]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.007778,0.007778,0.007778,0.93,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
1,img_14887.jpg,0.007778,0.007778,0.007778,0.93,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
2,img_62885.jpg,0.007778,0.007778,0.007778,0.93,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
3,img_45125.jpg,0.007778,0.007778,0.007778,0.93,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
4,img_22633.jpg,0.007778,0.007778,0.007778,0.93,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778


In [27]:
submission.to_csv("submission3.csv", index=False)

In [28]:
FileLink("submission3.csv")

/home/ubuntu/nbs/lesson2/submission3.csv